In [34]:
import attr
import os
import ujson
import bz2
import random

from glob import glob
from tqdm import tqdm
from multiprocessing import Pool
from itertools import islice
from functools import partial

from IPython.display import display, Markdown

In [2]:
def zip_offset(seq):
    """Yield (item, 0-1 offset).
    """
    size = len(seq)
    for i, item in enumerate(seq):
        offset = i / (size - 1) if (size - 1) else 0
        yield item, offset

In [3]:
def map_segment(func, path):
    """Parse JSON segment, apply worker function.
    """
    results = []
    
    with bz2.open(path) as fh:
        for line in fh:
            results.append(func(ujson.loads(line)))
            
    return results

In [8]:
@attr.s
class Corpus:
    
    root = attr.ib()
    
    def paths(self):
        return glob(os.path.join(self.root, '*.bz2'))
                        
    def map_novels(self, func, shuffle=True):
        """Apply a worker to segment files in parallel.
        """
        paths = self.paths()
        
        if shuffle:
            random.shuffle(paths)
        
        with Pool() as p:
            
            worker = partial(map_segment, func)
            
            for results in p.imap_unordered(worker, self.paths()):
                yield from results

In [5]:
corpus = Corpus('../../data/chicago-bins-tokens.json/')

In [27]:
def kwic_worker(kw, n, o1=0, o2=1, r=5):
    """KWIC matches, inside offset region.
    """
    matches = []
    for i, (t, offset) in enumerate(zip_offset(n['tokens'])):
        if o1 < offset < o2 and t.lower() == kw:

            cl = n['tokens'][i-r:i]
            cr = n['tokens'][i+1:i+r+1]

            matches.append((cl, t, cr))

    return matches

In [28]:
def kwic(kw, num_novels=1000, **kwargs):
    
    worker = partial(kwic_worker, kw, **kwargs)
    res_iter = islice(corpus.map_novels(worker), num_novels)
    
    matches = [m for ms in tqdm(res_iter) for m in ms]
    random.shuffle(matches)
    
    return matches

In [29]:
a0 = kwic('a', o1=0, o2=0.05)

1000it [00:29, 33.52it/s]


In [30]:
a1 = kwic('a', o1=0.95, o2=1)

1000it [00:28, 35.43it/s]


In [31]:
the0 = kwic('the', o1=0, o2=0.05)

1000it [00:32, 30.97it/s]


In [33]:
the1 = kwic('the', o1=0.95, o2=1)

1000it [00:28, 35.24it/s]


In [40]:
def display_kwic(matches):
    for cl, m, cr in matches:
        md = f'{" ".join(cl)} **{m}** {" ".join(cr)}'
        display(Markdown(md))

# "a" ~ beginning

In [42]:
display_kwic(a0[:200])

dusty stuffed fox , without **a** case , whose tail some

firm displayed much interest in **a** University of Georgia Law School

in confronting hers . Not **a** man to let his dignity

signs , she would have **a** strange and intriguing career ,

replied ; “ you got **a** kind heart , buddy .

...

- three years – was **a** nightmare of gasoline reek ,

look striking on you . **A** thin one , like Half

other side of the corral **a** pony snuffled at the hay

n't asking your daughter for **a** date , man . "

something,’ said Emily , after **a** pause . She spoke without

# "the" ~ beginning

In [44]:
display_kwic(the0[:200])

beard tried vainly to cover **the** pimples that were still on

been a great feeling during **the** few seconds he had shared

. ” Seth jumped into **the** helicopter and set the pup

green - brier grew in **the** corners , the prickly stems

put him off , but **the** eccentric dress of religion was

...

. ” “ He mentioned **the** virtues of the personal approach

took the sparkle away from **the** park . His depression was

I saw made me flee **the** house then , and Mother

- clad men disappearing in **the** club entrance as in a

If I wake up in **the** early morning . It ’s

# "a" ~ end

In [43]:
display_kwic(a1[:200])

standing at opposite sides of **a** doorway , leaning against the

overhead . It blew up **a** quarter acre of ice to

, Deborah . All of **a** sudden , I ’m waking

” The next voice was **a** lot like Barbara ’s ,

accompli . I shall send **a** notice of our engagement to

...

the dinner was off to **a** rollicking start . First Qwilleran

call the ow ! —Just **a** small needle lie still sir

just one damn reporter with **a** bunch of damn fool questions

matter , maybe there ’s **a** parallel world so glorious we

grin on my face and **a** sort of gut feeling that

# "the" ~ end

In [45]:
display_kwic(the1[:200])

“ Yes . ” “ **The** Congressional Medal of Honor ?

keep up , and all **the** citizens gaped and babbled in

, on both sides of **the** road . All those hundreds

club originated . Micrographs determining **the** temperature at which the torching

headed by Jameson climbed upon **the** Heiau . With crowbars they

...

, as if he knows **the** boy very well . Sonny

stationers’ books , something all **the** booksellers did two or three

home and deposited it in **the** Harpersville bank , something they

wish I had stamped on **the** lot of them ! ”

sent Carl to you in **the** hope that you could help